In [1]:
## Necessary Imports
import tensorflow as tf
import numpy as np
import scipy.io

In [2]:
## Mean values of the image taken for subtraction during preprocessing
IMAGE_MEAN = np.array([123.68 ,  116.779,  103.939])

In [8]:
##Defining the VGG Deep Neural Network
'''
Paraneters:
1. path_to_vgg_net : Location of the Pretrained VGG Deep Neural Network
2. input_image : image data

Getting into the layers,
conv are the convolution layers
relu is the relu activation function at the end of the convolution layer

Function Definitions:
1. scipy.io.loadmat(file_name, mdict=None, appendmat=True, **kwargs)
    Load MATLAB file.
2. enumerate:
    Return an enumerate object.  iterable must be another object that supports
    iteration.
'''
def net(path_to_vgg_net, input_image):

    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',

        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )

    data = scipy.io.loadmat(path_to_vgg_net)
    weights = data['layers'][0]

    net = {}
    current = input_image
    for i, name in enumerate(layers):
        layer_type = name[:4]
        if layer_type == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif layer_type == 'relu':
            current = tf.nn.relu(current)
        elif layer_type == 'pool':
            current = _pool_layer(current)
        net[name] = current

    return net

In [9]:
## Defining the convolution layer
'''
Parameters:
1. weights: A weight decides how much influence the input will have on the output.
2. bias: It is an additional parameter in the Neural Network which is used to adjust the output
         along with the weighted sum of the inputs to the neuron.
         
Function Definitions:
1. tf.nn.conv2d: Computes a 2-D convolution given 4-D `input` and `filter` tensors.
2. tf.nn.bias_add: Adds `bias` to `value`
'''

def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1), padding='SAME')
    return tf.nn.bias_add(conv, bias)

In [10]:
## Dening the Pooling layer
# Here we use Max Pool with 'SAME' padding
'''
Function Definitions:
1. tf.nn.max_pool: 
    Performs the max pooling on the input.

    Args:
      value: A 4-D `Tensor` with shape `[batch, height, width, channels]` and
        type `tf.float32`.
      ksize: A list of ints that has length >= 4.  The size of the window for
        each dimension of the input tensor.
      strides: A list of ints that has length >= 4.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. The padding algorithm.
        See the @{tf.nn.convolution$comment here}
      data_format: A string. 'NHWC' and 'NCHW' are supported.
      name: Optional name for the operation.

    Returns:
      A `Tensor` with type `tf.float32`.  The max pooled output tensor.

'''
def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME')

In [6]:
## Function to preprocess the image
def preprocess(image):
    return image - IMAGE_MEAN